In [1]:
import pandas as pd
import os
import math
import Bio
from Bio import Seq
import re
import matplotlib.pyplot as plt
import numpy as np
import Bio.Data.CodonTable
table = Bio.Data.CodonTable.ambiguous_dna_by_id[1]
import glob

In [2]:
dicSites={}
for i in glob.iglob(r'*/*coordinates*'):
    print(i)
    gene=i.split('/')[0]
    dicSites[gene]=pd.read_csv(i,header=0, index_col='NC_045512',sep='\t')

RdRp/RdRp_coordinates_conversion.tab
3CLpro/3CLpro_coordinates_conversion.tab
N/N_coordinates_conversion.tab
S/S_coordinates.tab
ORF1ab/ORF1ab_coordinates.tab
M/M_coordinates_conversion.tab
ORF3/ORF3_coordinates.tab
helicase/helicase_coordinates.tab


In [390]:
dicVar={}
for i in glob.iglob(r'*/*variant*'):
    print(i)
    gene=i.split('/')[0]
    dicVar[gene]=pd.read_csv(i,header=0, sep='\t')
    if 'MainResidue' in dicVar[gene].columns : 
        dicVar[gene]=dicVar[gene].set_index('MainResidue')


RdRp/RdRp_variants.tab
3CLpro/Annotation_variants_3C.tab
S/Annotation_variants_S.tab
nsp8/nsp8_variants.tsv
nsp7/nsp7_variants.tsv


In [391]:
dicSites.keys()

dict_keys(['RdRp', '3CLpro', 'N', 'S', 'ORF1ab', 'M', 'ORF3', 'helicase'])

In [392]:
dicSites['RdRp']

,AA,Alignment,NP_598309,AMM70617,AJ271965,KU740200,NP_828869,QEY10680,NC_045512.1,AID16548,AID16626,NC_019843,NP_740629,QDL88266,AIW52770,AID16752,NP_839965
NC_045512,,,,,,,,,,,,,,,,,
1,S,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0
2,A,7,1,0,1,1,2,1,2,1,1,1,7,1,1,2,1
3,D,8,2,0,2,2,3,2,3,2,2,2,8,2,2,3,2
4,A,9,3,0,3,3,4,3,4,3,3,3,9,3,3,4,3
5,Q,10,4,0,4,4,5,4,5,4,4,4,10,4,4,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928,H,946,923,0,924,929,928,924,928,923,924,929,936,924,923,928,923
929,T,947,924,0,925,930,929,925,929,924,925,930,937,925,924,929,924
930,V,948,925,0,926,931,930,926,930,925,926,931,938,926,925,930,925


In [393]:
dicVar['RdRp'].index

Int64Index([  5,   6,  10,  22,  23,  25,  44,  55,  63,  63,
            ...
            836, 848, 863, 876, 876, 879, 897, 916, 923, 928],
           dtype='int64', name='MainResidue', length=109)

In [394]:
dicResAnnot={}
common=[value for value in dicVar.keys() if value in dicSites.keys()] 
for i in dicSites.keys():
    dicResAnnot[i]=dicSites[i].copy()
    dicResAnnot[i]['Variant']='no'
    dicResAnnot[i]['Annotation']=np.nan
    dicResAnnot[i]['Impact']=np.nan
    dicResAnnot[i]['Reference']=np.nan
    
for i in common:
    for j,row in dicSites[i].iterrows() :
        if j in dicVar[i].index :
            dicResAnnot[i]['Variant']='no'
            if isinstance(dicVar[i].loc[j,], pd.core.frame.Series) :
                dicResAnnot[i].loc[j,'Annotation']=dicVar[i].loc[j,'Annotation']
                dicResAnnot[i].loc[j,'Impact']=dicVar[i].loc[j,'Impact']
                dicResAnnot[i].loc[j,'Reference']=dicVar[i].loc[j,'Reference']
            else :
                dicResAnnot[i].loc[j,'Annotation']=', '.join( list( dict.fromkeys(dicVar[i].loc[j,'Annotation'].dropna())))
                dicResAnnot[i].loc[j,'Impact']=', '.join(list( dict.fromkeys(dicVar[i].loc[j,'Impact'].dropna())))
                dicResAnnot[i].loc[j,'Reference']=', '.join(list( dict.fromkeys(dicVar[i].loc[j,'Reference'].dropna())))
    dicResAnnot[i]=dicResAnnot[i].replace(np.nan,'')


In [395]:
dicResAnnot.keys()

dict_keys(['RdRp', '3CLpro', 'N', 'S', 'ORF1ab', 'M', 'ORF3', 'helicase'])

In [396]:
dicResAnnot['S']

,AA,Alignment,FJ647223,JX504050,NC_038294,NC_004718,FJ647218,MN996527,NC_028806,AY714217,...,NC_038861,NC_002306,MG772934,AY463059,MN996528,DQ022305,Variant,Annotation,Impact,Reference
NC_045512,,,,,,,,,,,,,,,,,,,,,
1,M,8,2,0,5,1,2,1,7,1,...,4,8,2,1,1,5,no,,,
2,F,9,3,0,6,2,3,2,8,2,...,5,9,3,2,2,6,no,,,
3,V,11,5,3,7,3,4,3,9,3,...,6,10,4,3,3,7,no,,,
4,F,12,6,4,8,4,5,4,10,4,...,7,11,5,4,4,8,no,,,
5,L,13,7,5,9,5,6,5,11,5,...,8,12,6,5,5,9,no,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,K,1755,1359,1354,1351,1251,1320,1269,1380,1251,...,1445,1450,1241,1251,1269,1238,no,,,
1270,L,1756,1360,1355,1352,1252,1321,1270,1381,1252,...,1446,1451,1242,1252,1270,1239,no,,,
1271,H,1757,1361,1356,1353,1253,1322,1271,1382,1253,...,1447,1452,1243,1253,1271,1240,no,Cytoplasm Domain,,


In [397]:
sel_var=pd.read_csv('Selection_Variants.tsv',sep='\t', header=0)

In [398]:
[x for x in sel_var.ThirdFeat.unique() if x!='na']

[nan, 'ORF1ab']

In [399]:
[value for value in sel_var.ThirdFeat.unique() if value in dicSites.keys()] 

['ORF1ab']

In [400]:
allfeatures=list(sel_var.ThirdFeat.unique())+list(sel_var.SecondFeat.unique())+list(sel_var.MainFeat.unique())

In [401]:
[ x for x in allfeatures if x in dicResAnnot.keys()]

['ORF1ab', 'RdRp', 'helicase', 'S', 'M', 'N']

## Build Tables for genes with no coord conversion

In [355]:
import os
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import SeqIO



In [242]:
recs = [rec for rec in SeqIO.parse("/Users/delphinel/Work_dir/Covid/Ref_data/annotations.gb", "genbank")]
features_list=[{'gene': 'ORF1b polyprotein', 'start': 13467, 'end': 21555}]
for rec in recs:
    feats = [feat for feat in rec.features if feat.type == "mat_peptide" or feat.type == "CDS"]
    for feat in feats:
        product=feat.qualifiers['product'][0]
        start=int(feat.location.start)
        end=int(feat.location.end)
        res={'gene':product,'start':start,'end':end}
        features_list.append(res)
    feats = [feat for feat in rec.features if feat.type == "CDS"]
    for feat in feats:
        product=feat.qualifiers['product'][0]
        start=int(feat.location.start)
        end=int(feat.location.end)
        res={'gene':product,'start':start,'end':end}
        features_list.append(res)

In [244]:
features_list=[{'gene': 'ORF1b polyprotein', 'start': 13467, 'end': 21555},{'gene': 'RdRp1', 'start': 13441, 'end': 13468},{'gene': 'RdRp2', 'start': 13467, 'end': 16236}]
for rec in recs:
    feats = [feat for feat in rec.features if feat.type == "mat_peptide" or feat.type == "CDS"]
    for feat in feats:
        product=feat.qualifiers['product'][0]
        start=int(feat.location.start)
        end=int(feat.location.end)
        res={'gene':product,'start':start,'end':end}
        features_list.append(res)
    feats = [feat for feat in rec.features if feat.type == "CDS"]
    for feat in feats:
        product=feat.qualifiers['product'][0]
        start=int(feat.location.start)
        end=int(feat.location.end)
        res={'gene':product,'start':start,'end':end}
        features_list.append(res)


In [245]:
features_mat=pd.DataFrame(features_list)

In [246]:
features_mat=features_mat.drop_duplicates().sort_values(by=['start']).reset_index()[['gene','start','end']]

In [247]:
features_mat['short']='na'
for i,row in features_mat.iterrows():
    features_mat.loc[i, 'short']=row['gene'].split(' ')[0]



---

---

In [403]:
others=[x for x in allfeatures if x in list(features_mat.gene) or x in list(features_mat.short)] 

In [404]:
features_mat[(features_mat.gene=="3'-to-5' exonuclease") | (features_mat.short=="3'-to-5' exonuclease")]

,gene,start,end,short
18,3'-to-5' exonuclease,18039,19620,3'-to-5'


In [405]:
dicSize={}
for i in others:
    feat=features_mat[(features_mat.gene==i) | (features_mat.short==i)]
    size=(feat.end-feat.start)/3
    dicSize[i]=int(size)

In [406]:
dicSize

{'ORF1ab': 7096,
 'leader protein': 180,
 'nsp2': 638,
 'nsp3': 1945,
 'nsp4': 500,
 '3C-like proteinase': 306,
 'nsp6': 290,
 'nsp7': 83,
 'nsp8': 198,
 'nsp9': 113,
 'nsp10': 139,
 'helicase': 601,
 "3'-to-5' exonuclease": 527,
 'endoRNAse': 346,
 "2'-O-ribose methyltransferase": 298,
 'ORF1a': 4406,
 'ORF1b': 2696,
 'ORF3a': 276,
 'ORF6': 62,
 'ORF7a': 122,
 'ORF8': 122}

In [407]:
dicResAnnot.keys()

dict_keys(['RdRp', '3CLpro', 'N', 'S', 'ORF1ab', 'M', 'ORF3', 'helicase'])

In [408]:
for i in dicSize.keys():
    sites=range(1,7096)
    dicResAnnot[i]=pd.DataFrame(sites,columns =['NC_045512']) 
    dicResAnnot[i]['Annotation']=''
    dicResAnnot[i]['Impact']=''
    dicResAnnot[i]['Reference']=''
    dicResAnnot[i]=dicResAnnot[i].set_index('NC_045512')

In [409]:
dicResAnnot.keys()

dict_keys(['RdRp', '3CLpro', 'N', 'S', 'ORF1ab', 'M', 'ORF3', 'helicase', 'leader protein', 'nsp2', 'nsp3', 'nsp4', '3C-like proteinase', 'nsp6', 'nsp7', 'nsp8', 'nsp9', 'nsp10', "3'-to-5' exonuclease", 'endoRNAse', "2'-O-ribose methyltransferase", 'ORF1a', 'ORF1b', 'ORF3a', 'ORF6', 'ORF7a', 'ORF8'])

In [410]:
dicResAnnot['ORF1b']

,Annotation,Impact,Reference
NC_045512,,,
1,,,
2,,,
3,,,
4,,,
5,,,
...,...,...,...
7091,,,
7092,,,
7093,,,


In [411]:
sel_var

,Gene,POS,Genomic_coord_start,Genomic_coord_end,Main_ref_codon,Main_alt_codon,Main_ref_residue,Main_alt_residue,NbSample,MeanAF,...,ThirdFeat,ThirdResidue,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,ORF1ab pos,Annotation,Impact,Reference
0,"ORF1a 16,leader protein 16",311.0,311,313,"CTC,CTT","CTT,NNN,CTY",L,"L,X",2.0,0.0,...,na,na,na,na,na,na,16.0,leader protein,NaN,NaN
1,"ORF1a 30,leader protein 30",353.0,353,355,0,0,G,D,12.0,0.0,...,na,na,na,na,na,na,30.0,leader protein,NaN,NaN
2,"ORF1a 84,leader protein 84",515.0,515,517,GTT,"NNN,GTG,GNT,GNN",V,"X,V",2.0,0.0,...,na,na,na,na,na,na,84.0,leader protein,NaN,NaN
3,"ORF1a 86,leader protein 86",521.0,521,523,"GTT,ATG","ATG,NNN,NNT","V,M","M,X",2.0,0.0,...,na,na,na,na,na,na,86.0,NaN,NaN,NaN
4,"ORF1a 92,leader protein 92",539.0,539,541,"CTT,CTC","CTC,MTC,NNN,NTC,CYC,TTC,CTG",L,"L,J,X,F",2.0,0.0,...,na,na,na,na,na,na,92.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,N 393,29450.0,29450,29452,ACT,ATT,T,I,5.0,0.0,...,na,na,na,na,na,na,NaN,"B cell epitope region, Phosphorylation site in...",NaN,"10.1016/j.chom.2020.03.002, 10.1007/s11373-005..."
285,N 397,29462.0,29462,29464,"GCT,GTT","GTT,TCT,GNT","A,V","V,S,X",2.0,0.0,...,na,na,na,na,na,na,NaN,B cell epitope region,NaN,10.1016/j.chom.2020.03.002
286,N 399,29468.0,29468,29470,0,0,D,Y,3.0,0.0,...,na,na,na,na,NaN,NaN,NaN,B cell epitope region,NaN,10.1016/j.chom.2020.03.002
287,N 402,29477.0,29477,29479,0,0,D,Y,6.0,0.0,...,na,na,na,na,NaN,NaN,NaN,NaN,NaN,NaN


In [412]:
tmp=sel_var[(sel_var.ThirdFeat=='S') | (sel_var.SecondFeat=='S') | (sel_var.MainFeat=='S') ]

In [416]:
separate_variants={}

In [417]:
for i in dicResAnnot.keys():
    separate_variants[i]= sel_var[(sel_var.ThirdFeat==i) | (sel_var.SecondFeat==i) | (sel_var.MainFeat==i) ]

In [418]:
separate_variants.keys()

dict_keys(['RdRp', '3CLpro', 'N', 'S', 'ORF1ab', 'M', 'ORF3', 'helicase', 'leader protein', 'nsp2', 'nsp3', 'nsp4', '3C-like proteinase', 'nsp6', 'nsp7', 'nsp8', 'nsp9', 'nsp10', "3'-to-5' exonuclease", 'endoRNAse', "2'-O-ribose methyltransferase", 'ORF1a', 'ORF1b', 'ORF3a', 'ORF6', 'ORF7a', 'ORF8'])

In [419]:
dicResAnnot['ORF1a']

,Annotation,Impact,Reference
NC_045512,,,
1,,,
2,,,
3,,,
4,,,
5,,,
...,...,...,...
7091,,,
7092,,,
7093,,,


In [431]:

for i in separate_variants.keys():
    dicResAnnot[i]['Variant']='no'
    if i in list(sel_var.MainFeat):
        poscol='MainResidue'
    elif i in list(sel_var.SecondFeat):
        poscol='SecondResidue'
    elif i in list(sel_var.ThirdFeat):
        poscol='ThirdResidue'
    for j,row in separate_variants[i].iterrows() :
        dicResAnnot[i].loc[j,'Variant']='yes'
        dicResAnnot[i].loc[j,'Annotation']=separate_variants[i].loc[j,'Annotation']
        dicResAnnot[i].loc[j,'Impact']=separate_variants[i].loc[j,'Impact']
        dicResAnnot[i].loc[j,'Reference']=separate_variants[i].loc[j,'Reference']
    dicResAnnot[i]=dicResAnnot[i].replace(np.nan,'')

    

In [432]:
separate_variants['S']

,Gene,POS,Genomic_coord_start,Genomic_coord_end,Main_ref_codon,Main_alt_codon,Main_ref_residue,Main_alt_residue,NbSample,MeanAF,...,ThirdFeat,ThirdResidue,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,ORF1ab pos,Annotation,Impact,Reference
184,S 29,21647.0,21647,21649,ACT,"ATT,NCT",T,"I,X",4.0,0.0,...,na,na,na,na,na,na,NaN,N-terminal Domain,NaN,NaN
185,S 61,21743.0,21743,21745,AAT,AAC,N,N,2.0,0.0,...,na,na,na,na,na,na,NaN,N-terminal Domain,NaN,NaN
186,S 76,21788.0,21788,21790,0,0,T,I,6.0,0.0,...,na,na,na,na,NaN,NaN,NaN,N-terminal Domain,NaN,NaN
187,S 95,21845.0,21845,21847,0,0,T,I,6.0,0.0,...,na,na,na,na,NaN,NaN,NaN,N-terminal Domain,NaN,NaN
188,S 138,21974.0,21974,21976,GAT,"GAC,CAT,RAT,GAN,TAT",D,"D,H,B,X,Y",2.0,0.0,...,na,na,na,na,na,na,NaN,NaN,NaN,NaN
189,S 144,21992.0,21992,21994,0,0,V,D,2.0,0.0,...,na,na,na,na,NaN,NaN,NaN,NaN,NaN,NaN
190,S 231,22253.0,22253,22255,ATA,ACC,I,T,2.0,0.0,...,na,na,na,na,na,na,NaN,NaN,NaN,NaN
191,S 294,22442.0,22442,22444,GAC,"GAT,GNC,NNN,GAN",D,"D,X",2.0,0.0,...,na,na,na,na,na,na,NaN,NaN,NaN,NaN
192,S 367,22661.0,22661,22663,gtC,gtT,V,V,12.0,0.0,...,na,na,na,na,NaN,NaN,NaN,RBD,NaN,NaN
193,S 439,22877.0,22877,22879,0,0,N,K,4.0,0.0,...,na,na,na,na,NaN,NaN,NaN,"RBD, RBM, Variable residues between SARS-CoV-2...","Could impact Response to antibodies, Residue r...","10.1038/s41423-020-0400-4, 10.1038/s41467-020-..."


In [433]:
dicResAnnot['S'].Annotation.unique()

array(['', 'N-terminal Domain', 'RBD',
       'RBD, RBM, Variable residues between SARS-CoV-2 and SARS-CoV, critical residues in Polyclonal anti-SARS-1 antibodies and different between SARS-CoV-2 and SARS-CoV RBDs, ACE2-binding residue, SARS-CoV-derived discontinuous B cell epitopes associated with antibody m396',
       'HLA-DRB1*01:01, B cell epitope region',
       'B cell epitope region, North America Mutation, other mutations more likely to appear',
       'B cell epitope region', 'T Cell Epitopes',
       'T cell epitope, Epitope HLA locus A group 2 subtype 2:01, “HR1”, Fusion Core : \xa0target of fusion/entry inhibitors, One of the 8 residues that differs from SARS-Cov Fusion Core , position f in the heptad repeat abcdefg pattern, ',
       'HLA-B*15:01, HLA-B*27:05, HLA-B*58:01, HLA-C*15:02,  T Cell Epitopes',
       'HLA-DRB1*01:01',
       'RBD , Variable residues between SARS-CoV-2 and SARS-CoV',
       'RBD, residues fully conserved among SARS-CoV-2, SARS-CoV, and MERS-CoV'

In [434]:
dicResAnnot.keys()

dict_keys(['RdRp', '3CLpro', 'N', 'S', 'ORF1ab', 'M', 'ORF3', 'helicase', 'leader protein', 'nsp2', 'nsp3', 'nsp4', '3C-like proteinase', 'nsp6', 'nsp7', 'nsp8', 'nsp9', 'nsp10', "3'-to-5' exonuclease", 'endoRNAse', "2'-O-ribose methyltransferase", 'ORF1a', 'ORF1b', 'ORF3a', 'ORF6', 'ORF7a', 'ORF8'])

In [435]:
!mkdir residue_annotation

mkdir: residue_annotation: File exists


In [438]:
for i in dicResAnnot.keys():
    dicResAnnot[i].to_csv('residue_annotation/'+i+'_annot.tsv', sep='\t')

In [439]:
dicResAnnot

{'RdRp':           AA  Alignment  NP_598309  AMM70617  AJ271965  KU740200  NP_828869  \
 NC_045512                                                                     
 1          S          1          0         0         0         0          1   
 2          A          7          1         0         1         1          2   
 3          D          8          2         0         2         2          3   
 4          A          9          3         0         3         3          4   
 5          Q         10          4         0         4         4          5   
 ...       ..        ...        ...       ...       ...       ...        ...   
 928        H        946        923         0       924       929        928   
 929        T        947        924         0       925       930        929   
 930        V        948        925         0       926       931        930   
 931        L        949        926         0       927       932        931   
 932        Q        950        

In [451]:
dicfiles={}
for i in glob.iglob(r'residue_annotation/*'):
    name=i.replace('residue_annotation/','').replace('_annot.tsv','')
    dicfiles[name]=[name,'https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/'+i]

In [452]:
dicfiles

{'nsp10': ['nsp10',
  'https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/nsp10_annot.tsv'],
 'ORF1b': ['ORF1b',
  'https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/ORF1b_annot.tsv'],
 'helicase': ['helicase',
  'https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/helicase_annot.tsv'],
 'nsp3': ['nsp3',
  'https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/nsp3_annot.tsv'],
 'M': ['M',
  'https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/M_annot.tsv'],
 'ORF7a': ['ORF7a',
  'https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/ORF7a_annot.tsv'],
 'ORF1ab': ['ORF1ab',
  'https://raw.githubusercontent.com

In [453]:
tablefiles=pd.DataFrame.from_dict(dicfiles,orient='index', columns=['gene','url'])

In [454]:
tablefiles

,gene,url
nsp10,nsp10,https://raw.githubusercontent.com/galaxyprojec...
ORF1b,ORF1b,https://raw.githubusercontent.com/galaxyprojec...
helicase,helicase,https://raw.githubusercontent.com/galaxyprojec...
nsp3,nsp3,https://raw.githubusercontent.com/galaxyprojec...
M,M,https://raw.githubusercontent.com/galaxyprojec...
ORF7a,ORF7a,https://raw.githubusercontent.com/galaxyprojec...
ORF1ab,ORF1ab,https://raw.githubusercontent.com/galaxyprojec...
S,S,https://raw.githubusercontent.com/galaxyprojec...
nsp8,nsp8,https://raw.githubusercontent.com/galaxyprojec...
ORF6,ORF6,https://raw.githubusercontent.com/galaxyprojec...


In [457]:
tablefiles.to_csv('residue_annotation/accessurl.tsv', index=None, sep='\t')

In [458]:
!head accessurl.tsv



gene	url
nsp10	https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/nsp10_annot.tsv
ORF1b	https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/ORF1b_annot.tsv
helicase	https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/helicase_annot.tsv
nsp3	https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/nsp3_annot.tsv
M	https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/M_annot.tsv
ORF7a	https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/ORF7a_annot.tsv
ORF1ab	https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/ORF1ab_annot.tsv
S	https://raw.githubusercon